# MAL Using Annotation Types
* Image MAL with subclasses.
* This is the same task as the image mal tutorial but we are going to use annnotation types

In [ ]:
!pip install "labelbox[data]"

In [ ]:
# Run these if running in a colab notebook
COLAB = "google.colab" in str(get_ipython())
if COLAB:
    !git clone https://github.com/Labelbox/labelbox-python.git
    !mv labelbox-python/examples/model_assisted_labeling/image_model.py .
else:
    import sys
    sys.path.append('../model_assisted_labeling')

In [ ]:
#Used this as a reference for the model
#https://colab.research.google.com/github/tensorflow/tpu/blob/master/models/official/mask_rcnn/mask_rcnn_demo.ipynb#scrollTo=6lCL-ZcwaJbA
import numpy as np
from PIL import Image
import requests
import ndjson
import uuid
from io import BytesIO
import os
from getpass import getpass

from labelbox import (
    Client, 
    LabelingFrontend, 
    OntologyBuilder, 
    Tool, 
    Classification, 
    Option
)

from labelbox.data.annotation_types import (
    LabelList,
    ImageData,
    MaskData,
    Rectangle,
    ObjectAnnotation,
    ClassificationAnnotation,
    Point,
    ClassificationAnswer,
    Radio,
    Mask,
    Label
)
from labelbox.data.serialization import NDJsonConverter
from image_model import predict, class_mappings, load_model

In [ ]:
# If you don't want to give google access to drive you can skip this cell
# and manually set `API_KEY` below.
if COLAB:
    !pip install colab-env -qU
    from colab_env import envvar_handler
    envvar_handler.envload()

API_KEY = os.environ.get("LABELBOX_API_KEY")
if not os.environ.get("LABELBOX_API_KEY"):
    API_KEY = getpass("Please enter your labelbox api key")
    if COLAB:
        envvar_handler.add_env("LABELBOX_API_KEY", API_KEY)

In [ ]:
# Set this if running in colab. Otherwise it should work if you have the LABELBOX_API_KEY set.
API_KEY = os.environ["LABELBOX_API_KEY"]
# Only update this if you have an on-prem deployment
ENDPOINT = "https://api.labelbox.com/graphql"

In [ ]:
client = Client(api_key=API_KEY, endpoint=ENDPOINT)

In [ ]:
#Downloads weights and loads the model.
load_model()

## Create Predictions
* Create helper functions for processing the model outputs
* Make predictions
* Add predictions to a LabelList object

In [ ]:
def has_bag(person, bags):
    for bag in bags:
        if person.value.shapely.contains(bag.value.shapely.centroid):
            return True
    return False    

def get_annotations(boxes, classes, seg_masks):
    annotations = []
    for box, class_idx, seg in zip(boxes, classes, seg_masks):
        name = class_mappings[class_idx]
        value = None
        classifications = []
        if name in ['person', 'handbag']:
            value = Rectangle(
                start = Point(x = box[1], y = box[0]), end = Point(x = box[3], y = box[2])
            )
        elif name == 'car':
            value = Mask(mask = MaskData.from_2D_arr(arr = seg), color = (1,1,1))
        if value is not None:
            annotations.append(
                ObjectAnnotation(
                    name = name,
                    value = value
                )
            ) 
    return annotations

def update_bag_classifications(annotations):
    bags = [annot for annot in annotations if annot.name == 'handbag']
    people = [annot for annot in annotations if annot.name == 'person']
    for person in people:
        person.classifications = [ClassificationAnnotation(
            name = 'has_bag',
            value = Radio(answer = ClassificationAnswer(name = str(has_bag(person, bags))))
        )]


In [ ]:
# We can just start creating predictions whether or not we have a configured project in labelbox
image_urls = ['https://raw.githubusercontent.com/Labelbox/labelbox-python/develop/examples/assets/2560px-Kitano_Street_Kobe01s5s4110.jpg']

labellist = LabelList()
for image_url in image_urls:
    image_data = ImageData(url = image_url)
    height, width = image_data.value.shape[:2]
    prediction = predict(np.array([image_data.im_bytes]), min_score=0.5, height=height, width = width)
    annotations = get_annotations(prediction['boxes'], prediction['class_indices'], prediction['seg_masks'])
    update_bag_classifications(annotations)
    labellist.append(Label(
        data = image_data,
        annotations = annotations
    ))

## Project Setup
* Create project
* Use labellist.get_ontology() to automatically create the OntologyBuilder

In [ ]:
# Lets setup a project to label
# Note see Ontology, Project, and Project_setup notebooks for more information on this section.
project = client.create_project(name="mal_project")
dataset = client.create_dataset(name="mal_dataset")
editor = next(
    client.get_labeling_frontends(where=LabelingFrontend.name == 'editor'))
# Use the label collection to build the ontology
project.setup(editor, labellist.get_ontology().asdict())
project.datasets.connect(dataset)
project.enable_model_assisted_labeling()

## Prepare for upload
* Our local annotations need the following:
    1. signed url for segmentation masks
    2. data rows in labelbox
    3. feature schema ids

In [ ]:
signer = lambda _bytes: client.upload_data(content=_bytes, sign=True)
labellist.add_url_to_masks(signer) \
         .add_url_to_data(signer) \
         .assign_feature_schema_ids(OntologyBuilder.from_project(project)) \
         .add_to_dataset(dataset, signer)

## Convert to Prediction import format (NDJson)
* We want to create a json payload that matches this: https://docs.labelbox.com/data-model/en/index-en#annotations
* We can use the NDJsonConverter to turn our labellist containg predictions into ndjson

In [ ]:
ndjsons = list(NDJsonConverter.serialize(labellist))
print(ndjsons[0])

## Upload the annotations

In [ ]:
upload_task = project.upload_annotations(name=f"upload-job-{uuid.uuid4()}",
                                         annotations=ndjsons,
                                         validate=True)
# Wait for upload to finish
upload_task.wait_until_done()

In [ ]:
# Review the upload status
for status in upload_task.statuses[:5]:
    print(status)